In [1]:
import numpy as np
import os
import shutil
import glob
import subprocess
import sys
import cv2

In [3]:
# System alterations

# Set the print options to avoid truncation
np.set_printoptions(threshold=sys.maxsize) # Default threshold is 1000

In [2]:
BASE_DEST_DIR = os.path.join(os.sep + 'home', 'jason', 'Documents', 'repositories', 'DeepLabCut', 'examples', 'BiteCountA_pose')
MIDAS_BASE_DIR = os.getcwd()
MIDAS_INPUT_DIR = os.path.join(MIDAS_BASE_DIR, 'input')
MIDAS_OUTPUT_DIR = os.path.join(MIDAS_BASE_DIR, 'output')
SOURCE_DEST_DIR = os.path.join(BASE_DEST_DIR, 'extracted')
SOURCE2_DEST_DIR = os.path.join(BASE_DEST_DIR, 'video')

SUBSETS = ['train']
CLASSES = ['cow_bite']
SALIENTS = ['salient1', 'salient2']

# Pre-Test Depth Estimation

In [6]:
# Copy directories and files from video directory to input directory
os.makedirs(MIDAS_INPUT_DIR, exist_ok=True)
shutil.copytree(SOURCE2_DEST_DIR, MIDAS_INPUT_DIR, dirs_exist_ok=True)
print(f"Copied '{SOURCE_DEST_DIR}' to '{MIDAS_INPUT_DIR}'")

Copied '/home/jason/Documents/repositories/DeepLabCut/examples/BiteCountA_pose/extracted' to '/home/jason/Documents/repositories/MiDaS/input'


In [ ]:
# Extract frames from videos to images
video_files = glob.glob(os.path.join(MIDAS_INPUT_DIR, '**', '*.mp4'), recursive=True)
for video_file in video_files:
    video_dir = os.path.dirname(video_file)
    video_name = os.path.splitext(os.path.basename(video_file))[0]

    cap = cv2.VideoCapture(video_file)
    frame_num = 0
    while True:
        ret, frame = cap.read()
        if not ret:
            break

        image = os.path.join(video_dir, f"{video_name}_{frame_num}.jpg")
        cv2.imwrite(image, frame)
        frame_num += 1
    cap.release()

    print(f"Processed {frame_num} frames for video: {video_file}")

In [ ]:
# Remove video files in input directory
files = glob.glob(os.path.join(MIDAS_INPUT_DIR, '**', '*.mp4'), recursive=True)

# Delete non-JPG files
for file in files:
    if not file.endswith('.jpg') and not os.path.isdir(file):
        try:
            os.remove(file)
            print(f"Deleted: {file}")
        except Exception as e:
            print(f"Error deleting {file}: {e}")

In [ ]:
# Perform depth estimation on all images from pre_test extracted images
os.environ["MKL_THREADING_LAYER"] = "GNU"

video_dirs = glob.glob(os.path.join(MIDAS_INPUT_DIR, '*/'), recursive=True)
for video_dir in video_dirs:
    input_path = os.path.relpath(video_dir, MIDAS_BASE_DIR)
    output_path = os.path.relpath(video_dir.replace('input', 'output'), MIDAS_BASE_DIR)

    command = f'python run.py --model_type dpt_swin2_large_384 --input_path {input_path} --output_path {output_path}'
    subprocess.run(command, shell=True)

In [ ]:
# Move .pfm files to same location inside extracted directory
output_files = glob.glob(os.path.join(MIDAS_OUTPUT_DIR, '**', '*.pfm'), recursive=True)
for output_file in output_files:
    rel_path = os.path.relpath(output_file, MIDAS_OUTPUT_DIR)
    dest_file = os.path.join(SOURCE2_DEST_DIR, rel_path)

    destination_dir = os.path.dirname(dest_file)
    os.makedirs(destination_dir, exist_ok=True)

    shutil.move(output_file, dest_file)
    print(f"Moved {output_file} to {dest_file}")

# Pre-Train Depth Estimation

In [18]:
# Copy directories and files from extracted directory to input directory
os.makedirs(MIDAS_INPUT_DIR, exist_ok=True)
shutil.copytree(SOURCE_DEST_DIR, MIDAS_INPUT_DIR, dirs_exist_ok=True)
print(f"Copied '{SOURCE_DEST_DIR}' to '{MIDAS_INPUT_DIR}'")

Copied '/home/jason/Documents/repositories/DeepLabCut/examples/BiteCountA_pose/extracted' to '/home/jason/Documents/repositories/MiDaS/input'


In [ ]:
# Perform depth estimation on all images from pre_train extracted images
video_dirs = glob.glob(os.path.join(MIDAS_INPUT_DIR, '*/*/*/*/'), recursive=True)
for video_dir in video_dirs:
    input_path = os.path.relpath(video_dir, MIDAS_BASE_DIR)
    output_path = os.path.relpath(video_dir.replace('input', 'output'), MIDAS_BASE_DIR)

    command = f'python run.py --model_type dpt_swin2_large_384 --input_path {input_path} --output_path {output_path}'
    subprocess.run(command, shell=True)

In [ ]:
# Move .pfm files to same location inside extracted directory
output_files = glob.glob(os.path.join(MIDAS_OUTPUT_DIR, '**', '*.pfm'), recursive=True)
for output_file in output_files:
    rel_path = os.path.relpath(output_file, MIDAS_OUTPUT_DIR)
    dest_file = os.path.join(SOURCE_DEST_DIR, rel_path)

    destination_dir = os.path.dirname(dest_file)
    os.makedirs(destination_dir, exist_ok=True)

    shutil.copy2(output_file, dest_file)
    print(f"Moved {output_file} to {dest_file}")